<p><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/1e/UNAL_Logosimbolo.svg/583px-UNAL_Logosimbolo.svg.png" alt="" width="1280" height="300" /></p>

# AGGREGATIONS

In [0]:
from pyspark.sql import functions as fun

In [0]:
elements = [
    {"id": 1, "name": "July", "age": 34, "salary": 550, "role": "admin"},
    {"id": 1, "name": "July", "age": 34, "salary": 550, "role": "admin"},
    {"id": 2, "name": "Gabriel", "age": 29, "salary": 720, "role": "developer"},
    {"id": 3, "name": "Luis", "age": 42, "salary": 610, "role": "developer"},
    {"id": 4, "name": "John", "age": 51, "salary": 890, "role": "manager"},
    {"id": 5, "name": "Daniel", "age": 27, "salary": 480, "role": "developer"},
    {"id": 6, "name": "Mary", "age": 38, "salary": 700, "role": "admin"},
    {"id": 7, "name": "Monica", "age": 33, "salary": 460, "role": "tester"},
    {"id": 8, "name": "Andrea", "age": 45, "salary": 680, "role": "admin"},
    {"id": 9, "name": "Sebastian", "age": 31, "salary": 530, "role": "developer"},
    {"id": 10, "name": "Johana", "age": 26, "salary": 410, "role": "tester"},
    {"id": 11, "name": None, "age": 26, "salary": None, "role": "tester"},
    {"id": 12, "name": "Juan", "age": 45, "salary": 680, "role": None},
]
df = spark.createDataFrame(elements)
display(df)


## GROUP BY

### SIMPLE COLUMN

#### WAY 1

In [0]:
df.groupBy("role").count().display()

#### WAY 2

In [0]:
df.groupBy("role").agg(fun.count("*")).display()

### MULTI COLUMN

In [0]:
df.groupBy("role", "name").agg(fun.count("*").alias("counter")).display()

### MULTI AGGREGATION

In [0]:
df.groupBy("name").agg(
  fun.count("*").alias("counter"),
  fun.min("salary").alias("min_salary"),
  fun.max("salary").alias("max_salary"),
  fun.avg("salary").alias("avg_salary"),
  fun.countDistinct("salary").alias("counter_distinct")
  
).display()

### COLLECT

#### LIST

In [0]:
df.groupBy("role").agg(
    fun.collect_list("name").alias("unique_names")
).display()

#### SET

In [0]:
df.groupBy("role").agg(
    fun.collect_set("name").alias("unique_names")
).display()

### DICTIONARY

```python
join(
    other: 'DataFrame', 
    on: Union[str, List[str], 
    pyspark.sql.column.Column, 
    List[pyspark.sql.column.Column], NoneType] = None, 
    how: Optional[str] = None
)
```

In [0]:
df.groupBy("role").agg(
    {"salary": "avg", "age": "max"}
).display()

## JOINS

### DATASETS

In [0]:
elements_2 = [
{"id": 1, "name": "July", "age": 34, "salary": 550, "role": "admin", "document_type_id": 1, "active": "Y"},
    {"id": 1, "name": "July", "age": 34, "salary": 550, "role": "admin", "document_type_id": 1, "active": "Y"},
    {"id": 2, "name": "Gabriel", "age": 29, "salary": 720, "role": "developer", "document_type_id": 2, "active": "N"},
    {"id": 3, "name": "Luis", "age": 42, "salary": 610, "role": "developer", "document_type_id": 3, "active": "Y"},
    {"id": 4, "name": "John", "age": 51, "salary": 890, "role": "manager", "document_type_id": 5, "active": "Y"},  
    {"id": 5, "name": "Daniel", "age": 27, "salary": 480, "role": "developer", "document_type_id": None, "active": "N"}, 
    {"id": 6, "name": "Mary", "age": 38, "salary": 700, "role": "admin", "document_type_id": 2, "active": "Y"},
    {"id": 7, "name": "Monica", "age": 33, "salary": 460, "role": "tester", "document_type_id": None, "active": "N"},  
    {"id": 8, "name": "Andrea", "age": 45, "salary": 680, "role": "admin", "document_type_id": 6, "active": "Y"},  
    {"id": 9, "name": "Sebastian", "age": 31, "salary": 530, "role": "developer", "document_type_id": 1, "active": "Y"},
    {"id": 10, "name": "Johana", "age": 26, "salary": 410, "role": "tester", "document_type_id": 4, "active": "N"},
    {"id": 11, "name": None, "age": 26, "salary": None, "role": "tester", "document_type_id": None, "active": "N"}, 
    {"id": 12, "name": "Juan", "age": 45, "salary": 680, "role": None, "document_type_id": 3, "active": "Y"}
]

document_types = [
    {
        "document_type": "passport",
        "document_type_id": 1,
        "description": "International passport document",
        "active": "Y"
    },
    {
        "document_type": "national_id",
        "document_type_id": 2,
        "description": "National identity card",
        "active": "Y"
    },
    {
        "document_type": "driver_license",
        "document_type_id": 3,
        "description": "Driver's license document",
        "active": "N"
    },
    {
        "document_type": "residence_permit",
        "document_type_id": 4,
        "description": "Residency permit for foreign citizens",
        "active": "Y"
    }
]


document_typesV2 = [
    {
        "document_type": "passport",
        "type_id": 1,
        "description": "International passport document",
        "active": "Y"
    },
    {
        "document_type": "national_id",
        "type_id": 2,
        "description": "National identity card",
        "active": "Y"
    },
    {
        "document_type": "driver_license",
        "type_id": 3,
        "description": "Driver's license document",
        "active": "N"
    },
    {
        "document_type": "residence_permit",
        "type_id": 4,
        "description": "Residency permit for foreign citizens",
        "active": "Y"
    }
]


dfj = spark.createDataFrame(elements_2)
dtp = spark.createDataFrame(document_types)
dtpV2 = spark.createDataFrame(document_typesV2)


#### DOCUMENT TYPE

In [0]:
display(dtp)

#### USERS

In [0]:
display(dfj)

#### DOCUMENT TYPE V2

In [0]:
dtpV2.display()

### INNER

![inner](https://res.cloudinary.com/pym/image/upload/c_scale,f_auto,q_auto,w_258/articles/2019/sql/INNER_JOIN)

#### SIMPLE

In [0]:
dfj.join(dtp, on="document_type_id", how="inner").display()

#### DIFFERENT NAMES


In [0]:
dfj.join(dtpV2, on=dfj.document_type_id==dtpV2.type_id, how="inner").display()

#### MULTIPLE COLUNS

##### LIST

In [0]:
# INNER JOIN dfj.document_type_id = dtp.document_type_id AND Dfj.active = dtp.active
dfj.join(dtp, on=["document_type_id", "active"], how="inner").display()

##### ATRIBUTES

In [0]:
dfj.join(
    dtpV2,
    on=(
        (dfj.document_type_id==dtpV2.type_id) & (dfj.active==dtpV2.active)
    ),
    how="inner"
).display()

In [0]:
abc = dfj.join(
    dtpV2,
    on=(
        (dfj.document_type_id==dtpV2.type_id) & (dfj.active==dtpV2.active)
    ),
    how="inner"
)
abc.explain()

### LEFT

![inner](https://res.cloudinary.com/pym/image/upload/c_scale,f_auto,q_auto,w_258/articles/2019/sql/LEFT_JOIN)

In [0]:
dfj.join(dtp, on="document_type_id", how="left").display()

### RIGHT

![inner](https://res.cloudinary.com/pym/image/upload/c_scale,f_auto,q_auto,w_258/articles/2019/sql/RIGHT_JOIN)

In [0]:
dfj.join(dtp, on="document_type_id", how="right").display()

### OUTER

`outer` or `full `


![inner](https://res.cloudinary.com/pym/image/upload/c_scale,f_auto,q_auto,w_258/articles/2019/sql/FULL_JOIN)

In [0]:
dfj.join(dtp, on="document_type_id", how="full").display()

### CROSS

![inner|100x100](https://cdn.prod.website-files.com/676a9690ef4ec151a69571ff/67927323d9ca570e7ae71e8d_52018.svg)

In [0]:
dfj.crossJoin(dtp).display()

## UNION

In [0]:
dtpV2.dtypes

In [0]:
test = dtpV2.withColumnRenamed("type_id", "document_type_id")

### UNION

In [0]:
dtp.union(test).display()

### UNION ALL

In [0]:
dtp.unionAll(test).display()

### WITHOUT REPEATED VALUES

In [0]:
dtp.unionAll(test).distinct().display()

### UNION BY NAME

If they have a name, it is stricter

In [0]:
dtp.unionByName(test).display()